# volVal trial structure generation, radd simulation, and experimental results

In [ ]:
#write simulator for various trial structures
#function for trial structure, simulate results, then experimental 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random as rd
import itertools
from matplotlib import animation, rc
from IPython.display import HTML 
import csv
%matplotlib inline
import pandas as pd

In [ ]:
# def generateTrialStructure(nTrials, lambdaV, lambdaSS, ssd, sigmaMin, sigmaMax):
#     #insert code
#     return t1_baseReward, t2_baseReward, ssd, changePointVec, rewardDelta, 
#             muRewardDelta, sigmaDelta

In [237]:
#function inputs for testing 
nTrials = 1000
lambdaV = 10
lambdaStop = 10 #this will be constrained by the # of trials and perc. 
ssd = [200, 250, 300, 350, 450]

percStopTrials = .10
constantChangePoint = 0
muMin = .25
muMax = .75
sigmaMin = 0 
sigmaMax =  1 - muMax #needs to be chosen to ensure that r diff is !> 1 


In [5]:
#sample changePoint
changePoint_vec = np.zeros((nTrials, 1))
changePoints = []

if constantChangePoint == 0:
    
    i = 1
    while np.sum(changePoints) < nTrials:
        changePoints += list(np.random.poisson(lambdaV,i))   

    #if the sum(epochs) > nTrials, remove the last changePoint 
    #if we want to impose fewer constraints to prioritize integrity of 
    #random selection, we can relax the absolute # of trials 
    if np.sum(changePoints) > nTrials:
        changePoints = changePoints[0:-1]       
    
    #convert to trial number indices 
    changeIdx = list(itertools.accumulate(changePoints))
    #mark change point 
    changePoint_vec[changeIdx] = 1 
    
elif constantChangePoint == 1:
    changePoints = np.int(np.random.poisson(lambdaV,1))
    changePoint_vec[changePoints::changePoints,:] = 1 
# print(changeIdx)
# print(changePoint_vec)

In [ ]:
#stop trials 
nStopTrials = np.int(percStopTrials * nTrials)
stopPoints = np.random.poisson(lambdaStop,nStopTrials) #length
stopIdx = np.array(list(itertools.accumulate(stopPoints)))#indices

#this is not working
while np.sum(stopPoints) > nTrials: 
    #find location of samples > nTrials and resample 
    nStopRev = len(np.nonzero(stopIdx>nTrials))
    stopPoints[stopPoints>nTrials] = np.random.poisson(lambdaStop, nStopRev)
    stopIdx = np.array(list(itertools.accumulate(stopPoints)))
    check = len(np.unique(stopIdx)) == nStopTrials 
    
#assign delays to the stop indices 
nSamplesPerDelay = np.int(nStopTrials/len(ssd))
ssd_vec = np.zeros((nTrials, 1))
ssd_assignment = np.zeros((nSamplesPerDelay,1))

for idx in range(len(ssd)):
    ssd_assignment = np.random.choice(stopIdx, nSamplesPerDelay, replace = False)
    ssd_vec[ssd_assignment] = ssd[idx]
    stopIdx = stopIdx[stopIdx != ssd_assignment]
    print(stopIdx)

#     print(stopIdx)
#     print(len(stopIdx))

# print(len(ssd_vec[ssd_vec != 0]))
# print(stopIdx)
# print(stopIdx)
# print(check)


In [359]:
#means and sigma
nEpochs = np.int(np.sum(changePoint_vec)) + 1
mu_p = np.random.uniform(muMin, muMax, np.int(nEpochs/2))
mu_n = -1*mu_p
muRewardDelta = np.array(np.hstack((mu_p, mu_n)))
np.random.shuffle(muRewardDelta)
# print(nEpochs)

sigma = np.random.uniform(sigmaMin, sigmaMax, 1)

rewardDelta = np.zeros((nTrials,1))


test = changePoints[i]
        x = changePoints[i-1]
for i in range(0, nEpochs):
    for idx in range(x, test):
        rewardDelta[idx] = np.random.normal(muRewardDelta[idx], sigma)
     
# print(rewardDelta)
        print(i)

        #     np.random.normal(mu[], sigma[0],
# print(changePoints)

# print(rewardDelta)
#sample from a normal distribution to generate reward delta

IndentationError: unexpected indent (<ipython-input-359-055397588595>, line 14)

In [ ]:
#reverse engineer the actual values for targets

#assume baseline value of .5 for each
baseTargets = 0.5*np.ones((nTrials,2))
rewardDelta_div = rewardDelta / 2
adjustedTargets = np.ones_like(baseTargets)

#always add half and subtract half of target value
for i in range(0, len(baseTargets)):
    adjustedTargets[i,0] = baseTargets[i,0] +  rewardDelta_div[i]
    adjustedTargets[i,1] = baseTargets[i,1] -  rewardDelta_div[i]

print(adjustedTargets)


plt.title("raw target reward probabilities")
plt.xlabel("trial")
plt.ylabel("reward probability")
plt.legend(["t1", "t2"])
plt.plot(adjustedTargets[0:nTrials,0], 'b.')
plt.plot(adjustedTargets[0:nTrials,1], 'g.')


plt.title("difference between target reward probabilities")
plt.xlabel("trial")
plt.ylabel("$\Delta$ reward probability")
plt.legend(["t1", "t2"])
plt.plot(rewardDelta, 'k.')


# animate trial structure

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ax.set_xlim(( -2, 2))
ax.set_ylim((-2, 2))

sinegraph, = ax.plot([], [])
dot, = ax.plot([], [],'o', color='red')

x = np.linspace(0, 2, 1000)
y = np.cos(x)

In [ ]:
# initialization function: plot the background of each frame
def init():
    dot.set_data([], [])
    return (dot,)

In [ ]:
# animation function. This is called sequentially
def animate(i): 
#     y2 = np.ones(0, 2, 1000)
#     jitter = np.random.normal(0, 1, (100,1))
#     y=np.linspace(-1, 1, num=1000) * (x - 0.01 * i)
    sinegraph.set_data(x[:i],y[:i])
    dot.set_data(x[i],y[i])
    return (dot,)


In [ ]:
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=50, interval=10000, blit=True)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
#interface with radd kyle


## print parameters to a csv file 

In [ ]:
taskParameters = [t1_baseReward, t2_baseReward, ssd, changePointVec, rewardDelta, 
            muRewardDelta, sigmaDelta]
header = ("t1_baseReward, t2_baseReward, ssd, changePoint,
          "rewardDelta, muRewardDelta, sigmaDelta")
          
np.savetxt('VolVal_TrialStructure.csv',file_mat, header = header, delimiter=',', comments = '')